In [ ]:
%load_ext zipline

In [ ]:
import os
import pyfolio as pf
import pandas as pd
from zipline.utils.run_algo import load_extensions

load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)

In [ ]:
from zipline.data import bundles

bundle = bundles.load('eu_stocks_algo')
bundle.asset_finder.retrieve_all(bundle.asset_finder.sids)

In [ ]:
%%zipline --start 2018-1-1 --end 2018-12-31 --capital-base 1000.0 -o stochastic.pkl --bundle eu_stocks_algo --trading-calendar XAMS

# imports ----
from zipline.api import order_target, record, symbol, set_commission, cancel_order, order_percent
import matplotlib.pyplot as plt
import talib as ta
from zipline.finance import commission

# parameters ----
SELECTED_STOCK = 'AEX'

 #initialize the strategy 
def initialize(context):
    context.asset = symbol(SELECTED_STOCK)
    context.set_commission(commission.PerShare(cost=0.0, min_trade_cost=0))
    context.has_position = False
    
def handle_data(context, data):
    
    price_history = data.history(context.asset, fields="price", bar_count=34, frequency="1d")
    highs_history = data.history(context.asset, fields="high", bar_count=34, frequency="1d")
    lows_history = data.history(context.asset, fields="low", bar_count=34, frequency="1d")
    
    slowk,slowd = ta.STOCH(highs_history.values, lows_history.values, price_history.values, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    
    
    for i in range(1,len(slowk)):
        if (slowk[i-1] <= slowd[i-1]) and (slowk[i] >= slowd[i]) and not (context.has_position):
            order_percent(context.asset, 1)
            context.has_position = True
        elif (slowk[i-1] >= slowd[i-1]) and (slowk[i] <= slowd[i]) and (context.has_position):
            order_target(context.asset, 0)
            context.has_position = False
        else:
            order_target(context.asset, 0)
            context.has_position = False
            
        record(slowk =  slowk[-1], slowd = slowd[-1], price=price_history[-1])
        
  

In [ ]:
stochastic_results = pd.read_pickle('stochastic.pkl')

In [ ]:
import qf_helper_functions as qf
qf.visualize_results(stochastic_results, 'Stochastic Strategy - UNA', '€')

In [ ]:
stochastic_perf = qf.get_performance_summary(stochastic_results.returns)
print(stochastic_perf)

In [ ]:
perf_df = pd.DataFrame({'stochastic performance': stochastic_perf})
perf_df.transpose()

# Pyfolio Analysis

In [ ]:
returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(stochastic_results)

In [ ]:
returns.head()

In [ ]:
positions.head()

In [ ]:
transactions.head()

In [ ]:
pf.create_simple_tear_sheet(returns)

In [ ]:
pf.create_full_tear_sheet(returns)

# Single Plot Example

In [ ]:
pf.plot_drawdown_periods(returns, top=5).set_xlabel('Date')

# Full Tear Sheet Example

In [ ]:
pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions,
                          live_start_date='2018-1-5', round_trips=True)